# News Briefer

### Building News Briefer Using GloVe Algorithm

This notebook is for building a news briefer/summarizer. We would be scrapping TimesOfIndia RSS feeds to get news article text. This text is further passed to GloVe algorithm to create a brief summary of N sentences. 

Below is the basic data flow:
1. Get RSS feed for a topic. RSS feed contains news headline, link and published-date. 
2. Fetch the link HTML which contains the actual news article. 
3. Using BeautifulSoup library, extract news text from it. 
4. Perform basic text pre-processing such as cleanup/normalization
5. Generate sentence vectors using GloVe algorithm.
6. Rank the sentences using pagerank algorithm. 
7. Take top N sentences which forms the summary of the news article.



In [1]:
# import basic required libraries
import pandas as pd
import numpy as np
import os

# for web and HTML
import requests
from bs4 import BeautifulSoup


### Read the RSS feed

Let's first read the RSS feed and will pass it to BeautifulSoup for HTML parsing


In [3]:
# create a dict of various rss feed link and their categories. Will iterate them one by one.
# Have enabled only one feed to create a pipeline.
timesofindia = {'topstories':'https://timesofindia.indiatimes.com/rssfeedstopstories.cms',
                #'mostrecentstories':'https://timesofindia.indiatimes.com/rssfeeds/1221656.cms',
                #'world':'http://timesofindia.indiatimes.com/rssfeeds/296589292.cms'
               }
for category, rsslink in timesofindia.items():
    print('Processing for category: {0}. \nRSS link: {1}'.format(category,rsslink))
    
    # get the webpage URL and read the html
    print('Fetching for:',category)
    rssdata = requests.get(rsslink)
    print(rssdata.content)
#     soup = BeautifulSoup(data.content)
#     print('-----------------------')
#     print(soup)


Processing for category: topstories. 
RSS link: https://timesofindia.indiatimes.com/rssfeedstopstories.cms
Fetching for: topstories
b'<?xml version="1.0" encoding="UTF-8"?><rss xmlns:atom="http://www.w3.org/2005/Atom" version="2.0"><channel><atom:link type="application/rss+xml" rel="self" href="https://timesofindia.indiatimes.com/rssfeedstopstories.cms"/><title>Times of India</title><link>https://timesofindia.indiatimes.com</link><description>The Times of India: Breaking news, views, reviews, cricket from across India</description><language>en-gb</language><copyright>Copyright:(C) 2019 Bennett Coleman &amp; Co. Ltd, http://info.indiatimes.com/terms/tou.html</copyright><docs>http://timescontent.com/</docs><image><title>Times of India</title><link>https://timesofindia.indiatimes.com</link><url>https://timesofindia.indiatimes.com/photo.cms?msid=507610</url></image><item><title>Hyderabad encounter: Justice must never take form of revenge, says CJI</title><description>"Justice is never ough

Extracted HTML above looks raw and unformatted. Will pass it to BeautifulSoup library for parsing. 

In [4]:
# pass rssdata html to BeautifulSoup for parsing
soup = BeautifulSoup(rssdata.content)
# prettify() will give the visual representation of the parse tree created from the raw HTML content.
print(soup.prettify())


<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom">
   <channel>
    <atom:link href="https://timesofindia.indiatimes.com/rssfeedstopstories.cms" rel="self" type="application/rss+xml">
    </atom:link>
    <title>
     Times of India
    </title>
    <link/>
    https://timesofindia.indiatimes.com
    <description>
     The Times of India: Breaking news, views, reviews, cricket from across India
    </description>
    <language>
     en-gb
    </language>
    <copyright>
     Copyright:(C) 2019 Bennett Coleman &amp; Co. Ltd, http://info.indiatimes.com/terms/tou.html
    </copyright>
    <docs>
     http://timescontent.com/
    </docs>
    <image>
     <title>
      Times of India
     </title>
     <link/>
     https://timesofindia.indiatimes.com
     <url>
      https://timesofindia.indiatimes.com/photo.cms?msid=507610
     </url>
    </image>
    <item>
     <title>
      Hyderabad encounter: Justice must never take for

Before moving on, you should go through the HTML content of the webpage which we printed above using soup.prettify() method and try to find a pattern or a way to navigate to get the news headline, link and pubDate.


In [7]:
# get all news items. It has title, description, link, guid, pubdate for each news items. 
# Lets call this items and we will iterate thru it
allitems = soup.find_all('item')
print('Total news items found:',len(allitems))

# print one news item/healine to check
#for item in range(len(allitems)):
for item in range(1):
    print('Processing news-item #:',item)
    print(allitems[item])
    print('-------------------------')
    title = allitems[item].title.text
    link = allitems[item].guid.text
    pubdate = allitems[item].pubdate.text
    print('TITLE:',title)
    print('LINK:',link)
    print('PUBDATE:',pubdate)
    

Total news items found: 5
Processing news-item #: 0
<item><title>Hyderabad encounter: Justice must never take form of revenge, says CJI</title><description>"Justice is never ought to be instant. Justice must never ever take the form of revenge," said Chief Justice of India SA Bobde amid a raging debate on the justice delivery system in India over recent cases of rapes in Hyderabad and Unnao. "There is a need in judiciary to invoke self-correcting measures," he said adding that publicising them is a matter of debate.</description><link/>https://timesofindia.indiatimes.com/india/hyderabad-encounter-justice-must-never-take-form-of-revenge-says-cji-bobde/articleshow/72415223.cms<guid>https://timesofindia.indiatimes.com/india/hyderabad-encounter-justice-must-never-take-form-of-revenge-says-cji-bobde/articleshow/72415223.cms</guid><pubdate>Sat, 07 Dec 2019 10:56:42 GMT</pubdate></item>
-------------------------
TITLE: Hyderabad encounter: Justice must never take form of revenge, says CJI
LIN

News Title, link, and publish-date looks expected. Creating a basic function to fetch news text from link. Using BeautifulSoup, will extract news text present in between specific html tags. 


In [57]:
# Function to fetch each news link to get news essay 
def fetch_news_text(link):
    # read the html webpage and parse it
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    
    # fetch the news article text box
    # these are with element 
    # <div class="_3WlLe clearfix">
    text_box = soup.find_all('div', attrs={'class':'_3WlLe clearfix'})
    
    # Need to remove embeded link of other article. 
    # these are with element 
    # <div class="_3RArp undefined" data-type="embedgroup">
    remove_box = soup.find_all('div', attrs={'class':'_3RArp undefined','data-type':"embedgroup"})
    
    # PENDING: 
    # code to remove remove_box from text_box
    
    # extract text and combine
    news_text = str(". ".join(t.text.strip() for t in text_box))
    
    return news_text



Creating a loop to extract all news-stories in the RSS feed. 

In [58]:
news_articles = [{'Feed':'timesofindia',
                  'Category':category, 
                  'Headline':allitems[item].title.text, 
                  'Link':allitems[item].guid.text, 
                  'Pubdate':allitems[item].pubdate.text,
                  'NewsText': fetch_news_text(allitems[item].guid.text)} 
                     for item in range(5)]

news_articles = pd.DataFrame(news_articles)
news_articles


,Category,Feed,Headline,Link,NewsText,Pubdate
0,topstories,timesofindia,Hyderabad encounter: Justice must never take f...,https://timesofindia.indiatimes.com/india/hyde...,NEW DELHI: Amid a raging debate on the justice...,"Sat, 07 Dec 2019 10:56:42 GMT"
1,topstories,timesofindia,Rs 25L aid to Unnao victim's family: Key points,https://timesofindia.indiatimes.com/india/unna...,NEW DELHI: The rape victim from Uttar Prades...,"Sat, 07 Dec 2019 06:50:08 GMT"
2,topstories,timesofindia,Unnao rape case will be fast-tracked: Yogi,https://timesofindia.indiatimes.com/india/unna...,LUCKNOW: Uttar Pradesh chief minister Yogi Adi...,"Sat, 07 Dec 2019 05:08:02 GMT"
3,topstories,timesofindia,Judicial process beyond reach of poor: Kovind,https://timesofindia.indiatimes.com/india/judi...,JODHPUR: President Ram Nath Kovind here on Sat...,"Sat, 07 Dec 2019 11:48:07 GMT"
4,topstories,timesofindia,58.8% voting in Jharkhand till 3pm amid violence,https://timesofindia.indiatimes.com/india/58-8...,RANCHI: One person was killed as security pers...,"Sat, 07 Dec 2019 11:00:51 GMT"


### Text pre-processing (cleanup/normalization)

In [10]:
# import required libraries for text pre-processing
import spacy
import nltk
from nltk import sent_tokenize
from nltk.tokenize import ToktokTokenizer
import re
import unicodedata

from contractions import CONTRACTION_MAP # from contractions.py
nlp = spacy.load('en',parse=True,tag=True, entity=True) # required for lemmatization
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')


#### Removing HTML tags

Often, unstructured text contains a lot of noise, especially if you use techniques like web or screen scraping. HTML tags are typically one of these components which don’t add much value towards understanding and analyzing text.



In [16]:
# Removing HTML tags
def strip_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

strip_html_tags('<html><h2>First sentence of news article. And another one.</h2></html>')


'First sentence of news article. And another one.'

#### Removing accented characters

Usually in any text corpus, you might be dealing with accented characters/letters, especially if you only want to analyze the English language. Hence, we need to make sure that these characters are converted and standardized into ASCII characters. A simple example — converting é to e.



In [17]:
# removing accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

remove_accented_chars('Sómě Áccěntěd těxt.')


'Some Accented text.'

#### Expanding Contractions
Contractions are shortened version of words or syllables. They often exist in either written or spoken forms in the English language. These shortened versions or contractions of words are created by removing specific letters and sounds. In case of English contractions, they are often created by removing one of the vowels from the word. Examples would be, do not to don’t and I would to I’d. Converting each contraction to its expanded, original form helps with text standardization

We leverage a standard set of contractions available in the contractions.py

In [36]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all contractions expanded I'd think.")


'You all contractions expanded I would think.'

#### Removing Special Characters and numbers

Special characters and symbols are usually non-alphanumeric characters or even occasionally numeric characters (depending on the problem), which add to the extra noise in unstructured text. Usually, simple regular expressions (regexes) can be used to remove them.


In [40]:
# added .,?:;'" chars to retain
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9!.,?:;\'\"\s]' if not remove_digits else r'[^a-zA-z!.,?:;\'\"\s]'
    return re.sub(pattern, '', text)

remove_special_characters("Not sure if this was fun! What do you think of it.? 123#@!", 
                          remove_digits=True)

'Not sure if this was fun! What do you think of it.? !'

#### Stemming & Lemmatization

Get base word

In [50]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

simple_stemmer("My car keeps honking; my friend's honked yesterday.")


"My car keep honking; my friend' honk yesterday."

In [51]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text("My car keeps honking; my friend's honked yesterday.")


"My car keep honking ; my friend 's honk yesterday ."

#### Removing Stopwords
Words which have little or no significance, especially when constructing meaningful features from text, are known as stopwords or stop words. These are usually words that end up having the maximum frequency if you do a simple term or word frequency in a corpus. Typically, these can be articles, conjunctions, prepositions and so on. Some examples of stopwords are a, an, the, and the like.

In [52]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("The, and, if are stopwords, computer is not")


', , stopwords , computer not'

### Putting it together to make a text normalizer

In [53]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus


#### Check the normalizer function by passing one new's text

In [63]:
# test normalize cleanup on one article
normalize_corpus([news_articles['NewsText'][0]])

['new delhi : amid rage debate justice delivery system india recent incident rape hyderabad unnao , chief justice india sa bobde saturday say criminal justice system must reconsider position attitude warn justice never take form revenge . chief justice remark come day four accuse brutal rape murder case year old woman veterinarian hyderabad allegedly kill exchange fire police . " recent event country spark old debate new vigour , no doubt criminal justice system must reconsider position attitude towards time take dispose criminal matter , " say . however , chief justice caution justice must never ought instant . " justice never ought instant . justice must never ever take form revenge . believe justice lose character become revenge . need judiciary invoke self correct measure whether not publicise matter debate , " say . say need devise method not speed litigation prevent altogether . " law provide pre litigation mediation , " say , add need consider compulsory pre litigation mediation

## Vector representation of sentence

#### GloVe word embeddings
We will be using the pre-trained Wikipedia 2014 + Gigaword 5 GloVe vectors available here. Heads up – the size of these word embeddings is 822 MB.

In [64]:
# extract word vectors

# define dict to hold a word and its vector
word_embeddings = {}
# read the file
f = open('.\\GloVe\\glove.6B\\glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

len(word_embeddings)


400000

We have 400K word embedding. Now, let’s create vectors for our sentences. We will first fetch vectors (each of size 100 elements) for the constituent words in a sentence and then take mean/average of those vectors to arrive at a consolidated vector for the sentence.

Will follow below steps pipeline
1. create sentence from news article
2. clean each sentences
3. create vector for each sentences
4. create simillarity-matrix
5. Apply page-rank on simillarity-matrix
6. get top N sentences based on page-rank


In [69]:
# lets take one news-article to create pipeline. We will generalize it for complete corpus
# news_articles['CleanText'][2]

# 1. create sentences for each news-article
sentences = []

for s in normalize_corpus([news_articles['NewsText'][0]]):
#for s in [news_articles['NewsText'][2]]:
    sentences.append(sent_tokenize(s))

# flatten the list
sentences = [y for x in sentences for y in x]
print('Total number of sentence: {0}'.format(format(len(sentences))))

# 2. clean each sentences
clean_sentences = normalize_corpus(sentences)
print('Total cleaned sentence:',len(clean_sentences))

# 3. create vector for each sentences
# list to hold vector 
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

print('Total vectors created:',len(sentence_vectors))


Total number of sentence: 15
Total cleaned sentence: 15
Total vectors created: 15


#### Similarity Matrix Preparation
The next step is to find similarities between the sentences, and we will use the cosine similarity approach for this challenge. Let’s create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

Let’s first define a zero matrix of dimensions (n * n).  We will initialize this matrix with cosine similarity scores of the sentences. Here, n is the number of sentences.

We will use Cosine Similarity to compute the similarity between a pair of sentences.


In [70]:
# 4. create simillarity-matrix

# similarity matrix
# define matrix with all zero values
# will populate it with cosine_similarity values 
# for each sentences compared to other

sim_mat = np.zeros([len(sentences),len(sentences)])

from sklearn.metrics.pairwise import cosine_similarity

for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                              sentence_vectors[j].reshape(1,100))[0,0]


#### Apply PageRank Algorithm

Before proceeding further, let’s convert the similarity matrix sim_mat into a graph. The nodes of this graph will represent the sentences and the edges will represent the similarity scores between the sentences. On this graph, we will apply the PageRank algorithm to arrive at the sentence rankings.

In [71]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)


In [72]:
scores

{0: 0.06877575530748713,
 1: 0.06620696025988485,
 2: 0.06961142749476128,
 3: 0.06836042068386161,
 4: 0.06587982908448074,
 5: 0.06845233661633444,
 6: 0.06697288393169526,
 7: 0.06734312758532424,
 8: 0.06658546976256263,
 9: 0.06675300416936665,
 10: 0.06527808631852403,
 11: 0.06309025837987235,
 12: 0.06610562619331287,
 13: 0.06400456067973156,
 14: 0.06658025353280025}

### FINAL STEP:  Summary Extraction

Finally, it’s time to extract the top N sentences based on their rankings for summary generation.

In [73]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

# Extract top 10 sentences as the summary
for i in range(5):
    print(ranked_sentences[i][1])


recent event country spark old debate new vigour , no doubt criminal justice system must reconsider position attitude towards time take dispose criminal matter , " say .
new delhi : amid rage debate justice delivery system india recent incident rape hyderabad unnao , chief justice india sa bobde saturday say criminal justice system must reconsider position attitude warn justice never take form revenge .
justice must never ever take form revenge .
however , chief justice caution justice must never ought instant . "
need judiciary invoke self correct measure whether not publicise matter debate , " say .


# Bringing it all together — Building a News Briefer


In [74]:
# import basic required libraries
import pandas as pd
import numpy as np
import os

# for web and HTML
import requests
from bs4 import BeautifulSoup

# for text pre-processing
import spacy
import nltk
from nltk import sent_tokenize
from nltk.tokenize import ToktokTokenizer
import re
import unicodedata

from contractions import CONTRACTION_MAP # from contractions.py
nlp = spacy.load('en',parse=True,tag=True, entity=True) # required for lemmatization
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


In [75]:
# Function to fetch each news link to get news essay 
def fetch_news_text(link):
    # read the html webpage and parse it
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')
    
    # fetch the news article text box
    # these are with element 
    # <div class="_3WlLe clearfix">
    text_box = soup.find_all('div', attrs={'class':'_3WlLe clearfix'})
    
    # Need to remove embeded link of other article. 
    # these are with element 
    # <div class="_3RArp undefined" data-type="embedgroup">
    remove_box = soup.find_all('div', attrs={'class':'_3RArp undefined','data-type':"embedgroup"})
    
    # PENDING: 
    # code to remove remove_box from text_box
    
    # extract text and combine
    news_text = str(". ".join(t.text.strip() for t in text_box))
    
    return news_text


In [76]:
# custom function for normalizer
# Removing HTML tags
def strip_html_tags(text):
    #soup = BeautifulSoup(text,'html.parser')
    soup = BeautifulSoup(text)
    stripped_text = soup.get_text()
    return stripped_text

# removing accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

# remove special characters
# added .,?:;'" chars to retain
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9!.,?:;\'\"\s]' if not remove_digits else r'[^a-zA-z!.,?:;\'\"\s]'
    text = re.sub(pattern, '', text)
    return text

# stemmer
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# lemmatization
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

# remove stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


In [77]:
# function to clean and normalize text corpus
#def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
def normalize_corpus(doc, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    #normalized_corpus = []
    #normalized_doc = ''
    # normalize each document in the corpus
    #for doc in corpus:
    # strip HTML
    if html_stripping:
        doc = strip_html_tags(doc)
    # remove accented characters
    if accented_char_removal:
        doc = remove_accented_chars(doc)
    # expand contractions    
    if contraction_expansion:
        doc = expand_contractions(doc)
    # lowercase the text    
    if text_lower_case:
        doc = doc.lower()
    # remove extra newlines
    doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
    # lemmatize text
    if text_lemmatization:
        doc = lemmatize_text(doc)
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_characters(doc, remove_digits=remove_digits)  
    # remove extra whitespace
    doc = re.sub(' +', ' ', doc)
    # remove stopwords
    if stopword_removal:
        doc = remove_stopwords(doc, is_lower_case=text_lower_case)

    #normalized_corpus.append(doc)

    #return normalized_corpus
    return doc



In [5]:
# extract word vectors

# define dict to hold a word and its vector
word_embeddings = {}
# read the file
f = open('.\\GloVe\\glove.6B\\glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()


In [78]:
# function to summarize a corpus
def summarize(corpus,     # text corpus to summarize
              summ_cnt=5  # count of summary lines required
             ):
    
    # create sentences for each news-article
    # list to hold sentences
    sentences = []
    sentences.append(sent_tokenize(corpus))
    # flatten the list
    sentences = [y for x in sentences for y in x]
    #print('Total number of sentence: {0}'.format(format(len(sentences))))

    # skipping cleanup & normalize step because we already did in previous step
    # 2. clean & normalize each sentences
    # clean_sentences = normalize_corpus(sentences)
    # print('Total cleaned sentence:',len(clean_sentences))

    # 3. create vector for each sentences
    # list to hold vector 
    sentence_vectors = []
    #for i in clean_sentences:
    for i in sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    #print('Total vectors created:',len(sentence_vectors))

    # 4. create simillarity-matrix
    # similarity matrix and populate with cosine comparision values
    sim_mat = np.zeros([len(sentences),len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100),
                                                  sentence_vectors[j].reshape(1,100))[0,0]

    # Apply pagerank algorithm
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    # Extract top 10 sentences as the summary
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    summary = ''
    summ_cnt = summ_cnt if summ_cnt < len(sentences) else len(sentences)
    for i in range(summ_cnt):
        summary = summary + ranked_sentences[i][1]

    return summary



In [82]:
# create a dict of various rss feed link and their categories. Will iterate them one by one.
timesofindia = {'topstories':'https://timesofindia.indiatimes.com/rssfeedstopstories.cms',
                'mostrecentstories':'https://timesofindia.indiatimes.com/rssfeeds/1221656.cms',
                'world':'http://timesofindia.indiatimes.com/rssfeeds/296589292.cms'
               }

# create list to hold all news article details
news_articles = []

feed_name = 'timesofindia'
print('Feed:',feed_name)
for category, rsslink in timesofindia.items():
    # read HTML webpage from RSS URL
    print('- Fetching {0}.'.format(category))
    rssdata = requests.get(rsslink)
    
    # pass rssdata html to BeautifulSoup for parsing
    soup = BeautifulSoup(rssdata.content)
    
    # extract all news items. It has title, description, link, guid, pubdate for each news items. 
    allitems = soup.find_all('item')
    print(' - Found {0} items'.format(len(allitems)))
    
    for item in range(len(allitems)):
        print('  - Processing item #',item)
        headline = allitems[item].title.text
        link = allitems[item].guid.text
        pubdate = allitems[item].pubdate.text
        newstext = fetch_news_text(link)
        newstext_clean = normalize_corpus(newstext)
        newstext_smry = summarize(newstext_clean)
        
        # combine all parts into one list and append to main list
        this_item = [{'Feed':feed_name,
                      'Category':category, 
                      'Headline':headline, 
                      'Link':link, 
                      'Pubdate':pubdate,
                      'NewsText': newstext,
                      'NewstextClean':newstext_clean,
                      'NewstextSmry':newstext_smry,
                     }]
        news_articles.extend(this_item)

# create final dataframe
news_articles = pd.DataFrame(news_articles)
print('DONE')

Feed: timesofindia
- Fetching topstories.
 - Found 5 items
  - Processing item # 0
  - Processing item # 1
  - Processing item # 2
  - Processing item # 3
  - Processing item # 4
- Fetching mostrecentstories.
 - Found 20 items
  - Processing item # 0
  - Processing item # 1
  - Processing item # 2
  - Processing item # 3
  - Processing item # 4
  - Processing item # 5
  - Processing item # 6
  - Processing item # 7
  - Processing item # 8
  - Processing item # 9
  - Processing item # 10
  - Processing item # 11
  - Processing item # 12
  - Processing item # 13
  - Processing item # 14
  - Processing item # 15
  - Processing item # 16
  - Processing item # 17
  - Processing item # 18
  - Processing item # 19
- Fetching world.
 - Found 20 items
  - Processing item # 0
  - Processing item # 1
  - Processing item # 2
  - Processing item # 3
  - Processing item # 4
  - Processing item # 5
  - Processing item # 6
  - Processing item # 7
  - Processing item # 8
  - Processing item # 9
  - Pro

In [84]:
news_articles.head()

,Category,Feed,Headline,Link,NewsText,NewstextClean,NewstextSmry,Pubdate
0,topstories,timesofindia,UP govt announces Rs 25L financial aid for Unn...,https://timesofindia.indiatimes.com/india/unna...,NEW DELHI: The rape victim from Uttar Prades...,new delhi : rape victim uttar pradesh unnao di...,"no fear among criminal state , "" say meet vict...","Sat, 07 Dec 2019 06:50:08 GMT"
1,topstories,timesofindia,"Justice must never take form of revenge, says CJI",https://timesofindia.indiatimes.com/india/hyde...,NEW DELHI: Amid a raging debate on the justice...,new delhi : amid rage debate justice delivery ...,recent event country spark old debate new vigo...,"Sat, 07 Dec 2019 10:56:42 GMT"
2,topstories,timesofindia,Hyderabad encounter: NHRC team begins probe,https://timesofindia.indiatimes.com/city/hyder...,HYDERABAD: A seven-member NHRC team on Saturda...,hyderabad : seven member nhrc team saturday vi...,"however , political leader prominent people sp...","Sat, 07 Dec 2019 13:47:42 GMT"
3,topstories,timesofindia,Unnao rape case will be fast-tracked: Yogi,https://timesofindia.indiatimes.com/india/unna...,LUCKNOW: Uttar Pradesh chief minister Yogi Adi...,lucknow : uttar pradesh chief minister yogi ad...,unnao rape victim dad want hyderabad like puni...,"Sat, 07 Dec 2019 05:08:02 GMT"
4,topstories,timesofindia,Judicial process beyond reach of poor: Kovind,https://timesofindia.indiatimes.com/india/judi...,JODHPUR: President Ram Nath Kovind here on Sat...,jodhpur : president ram nath kovind saturday e...,"keep mind gandhiji famous criterion , remember...","Sat, 07 Dec 2019 11:48:07 GMT"


In [86]:
news_articles.columns

Index(['Category', 'Feed', 'Headline', 'Link', 'NewsText', 'NewstextClean',
       'NewstextSmry', 'Pubdate'],
      dtype='object')

In [88]:
news_articles['Category'].value_counts()

mostrecentstories    20
world                20
topstories            5
Name: Category, dtype: int64

## Congratulation, you have build a news briefer